<a href="https://colab.research.google.com/github/bernhardbrugger/bernhardbrugger/blob/main/Face_Gender_Age_Image_and_Video_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a short script that enables face detection and gender + age prediction using computer vision. 

In [ ]:
import numpy as np
import cv2

In [ ]:
# define the groups of ages which should be summarized in order to increase accuracy

AGE_GROUPS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]


# define the groups of genders which should be summarized in order to increase accuracy

GENDER_GROUPS = ["male", "female"]


# load the pretrained face detector model

PBTXT_PATH = r"..\opencv_face_detector.pbtxt"

PB_PATH = r"..\opencv_face_detector_uint8.pb"

face_architecture = cv2.dnn.readNet(PBTXT_PATH, PB_PATH)


# load the pretrained gender detector model

PBTXT_PATH = r"..\gender_deploy.prototxt"

CAF_PATH = r"..\gender_net.caffemodel"

gender_architecture = cv2.dnn.readNet(PBTXT_PATH, CAF_PATH)

In [ ]:
# this code captures the webcam of your laptop, detects your face and predicts your gender and age

video_capture = cv2.VideoCapture(0)

while(True):
    
    ret_val, frame = video_capture.read()
    
    frame = cv2.resize(frame, (600,400))
    
    (h, w) = frame.shape[:2]
    
    blob = cv2.dnn.blobFromImage(frame, 1.0, (227, 227), (104.0, 177.0, 123.0))
    
    
    face_architecture.setInput(blob)
    
    detections = face_architecture.forward()

    
    for i in range(0, detections.shape[2]):
    
        # get probabilities from the prediction
    
        confidence = detections[0, 0, i, 2]

    
        # filter low confidences
    
        # 0.5 is the minimum confidence
    
        if confidence > 0.5:


            # get the coordinates for the bounding boxes

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

            (startX, startY, endX, endY) = box.astype("int")
        


            '''# ensure the face ROI is sufficiently large

            if face.shape[0] < 20 or face.shape[1] < 20:

                continue'''


            # ROI for faces
 
            face = frame[startY:endY, startX:endX]
 
            blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)


    # age detector
 
    age_architecture.setInput(blob)
 
    predictions = age_architecture.forward()
 
    i = predictions[0].argmax()
 
    age = AGE_GROUPS[i]
 
    age_confidence = predictions[0][i]


    # gender detector

    gender_architecture.setInput(blob)

    predictions = gender_architecture.forward()

    i = predictions[0].argmax()

    gender = GENDER_GROUPS[i]

    gender_confidence = predictions[0][i]


    # show prediction in image

    txt = "gender: {}: {:.2f}%; age{}: {:.2f}%".format(gender, gender_confidence * 100, age, age_confidence * 100)


    # draw bounding box and age

    if startY - 10 > 10:

        y = startY - 10

    else:

        y = startY + 10


    cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)

    cv2.putText(frame, txt, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

            
    cv2.imshow("Frame", frame)


    if cv2.waitKey(1) & 0xFF == ord("q"):

        break


video_capture.release()

cv2.destroyAllWindows()

In [ ]:
# this code uploads a picture of your choice, detects face, predicts gender an age and returns the picture with the detected face and information about gender and age

# load and preprocess an image

IMAGE_PATH = r"..\mark_zuckerberg.jpg"

img = cv2.imread(IMAGE_PATH)

(h, w) = img.shape[:2]

blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), (104.0, 177.0, 123.0))


# pass the image through the network

face_architecture.setInput(blob)

detections = face_architecture.forward()


# get ROI coordinates

for i in range(0, detections.shape[2]):

    # get probabilities from the prediction

    confidence = detections[0, 0, i, 2]


    # filter low confidences

    # 0.5 is the minimum confidence

    if confidence > 0.5:


        # get the coordinates for the bounding boxes

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])

        (startX, startY, endX, endY) = box.astype("int")


        # ROI for faces

        face = img[startY:endY, startX:endX]

        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)


# age detector

age_architecture.setInput(blob)

predictions = age_architecture.forward()

i = predictions[0].argmax()

age = AGE_GROUPS[i]

age_confidence = predictions[0][i]


# gender detector

gender_architecture.setInput(blob)

predictions = gender_architecture.forward()

i = predictions[0].argmax()

gender = GENDER_GROUPS[i]

gender_confidence = predictions[0][i]


# show prediction in image

txt = "gender: {}: {:.2f}%; age{}: {:.2f}%".format(gender, gender_confidence * 100, age, age_confidence * 100)


# draw bounding box and age

if startY - 10 > 10:

    y = startY - 10

else:

    y = startY + 10


cv2.rectangle(img, (startX, startY), (endX, endY), (0, 0, 255), 2)

cv2.putText(img, txt, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)


# display output

cv2.imshow("Image", img)

cv2.waitKey(0)